In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import *
import numpy as np
import sys, os
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *
from _global.bbx import *
from _global.recognition import *
sys.path.extend([RESEARCH])
from object_detection.utils import config_util
from object_detection.builders import model_builder
from IPython import get_ipython
import cv2 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
ipython = get_ipython()

ModuleNotFoundError: No module named 'tensorflow_addons'

In [ ]:
def predict_face(frame, localizer, detector):
    face, best_bbx = find_and_extract_face(localizer, frame, None, True)
    face = tf.convert_to_tensor(face,dtype=tf.uint8)
    face = tf.expand_dims(face, 0)
    label = detector.predict(face)
    label = np.squeeze(label,axis=0)
    max_label = 0
    for i in range(len(label)):
        if label[i] >= label[max_label]:
            max_label = i

    #max label +2 because now we work with indecies because we have to work with ids that start from 1 and id 1 is unknown
    if label[max_label] < .60:
        max_label = 1
    else: max_label +=2

    label_map = label_map_util.load_labelmap(FACES_RECOG_LABEL_MAP)

    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=max_classes+1, use_display_name=True)

    category_index = label_map_util.create_category_index(categories)   

    x = np.array(1.0)

    img_with_bbxs = viz_utils.visualize_boxes_and_labels_on_image_array(frame,
                                                        boxes=np.array([best_bbx]),
                                                        classes=np.array([max_label]),
                                                        scores=np.array([1.0]),
                                                        category_index=category_index,
                                                        agnostic_mode=False,
                                                        skip_scores=True
                                                    )
    
    print(label)
    return img_with_bbxs


In [ ]:
face_recog = models.load_model(OUTPUT_FACES_RECOG)

config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
bbx_localizer = model_builder.build(model_config=config['model'], is_training=False)
ckpt = tf.compat.v2.train.Checkpoint(model=bbx_localizer) 
ckpt.restore(os.path.join(MODEL, 'ckpt-12')).expect_partial()


In [ ]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
while cap.isOpened():
    ret, frame = cap.read()

    if ret == False:
        continue

    image_np_with_detections = predict_face(frame, bbx_localizer, face_recog)
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (width, height)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break